In [ ]:
using Revise

In [ ]:
TE = joinpath(tempdir(), "Sarcopenia.test")

if isdir(TE)

    rm(TE, recursive = true)

    println("Removed ", TE, ".")

end

mkdir(TE)

println("Made ", TE, ".")

---

In [ ]:
using OnePiece

using GSEA

In [ ]:
se = joinpath(dirname(@__DIR__), "settings.json")

ke_ar = OnePiece.extension.dict.read(se)

In [ ]:
da = joinpath(@__DIR__, "sarcopenia")

readdir(da)

In [ ]:
ou = joinpath(TE, "sarcopenia")

;

In [ ]:
GSEA.standard(
    se,
    joinpath(da, "selected.set_to_genes.json"),
    joinpath(da, "target_by_sample.tsv"),
    joinpath(da, "gene_by_sample.tsv"),
    ou,
)

readdir(ou)

### Check set statistics

In [ ]:
na = "set_by_statistic.tsv"

ol_se_st = OnePiece.io.table.read(joinpath(da, string(ke_ar["permutation"], ".", na)))

ne_se_st = OnePiece.io.table.read(joinpath(ou, na))

se_ = ol_se_st[!, 1]

ne_se_st = ne_se_st[indexin(se_, ne_se_st[!, 1]), :]

for (nao, nan) in [
    #["ES", "Enrichment"],
    ["NOM p-val", "P-value"],
    #["FDR q-val", "Q-value"],
]

    display(
        OnePiece.figure.plot_x_y(
            [ol_se_st[!, nao]],
            [ne_se_st[!, nan]],
            text_ = [se_],
            mode_ = ["markers"],
            la = Dict(
                "xaxis" => Dict("title" => Dict("text" => nao)),
                "yaxis" => Dict("title" => Dict("text" => nan)),
            ),
        ),
    )

end

### Check signal-to-noise ratio

In [ ]:
na = "gene_by_statistic.tsv"

ol_ge_st = OnePiece.io.table.read(joinpath(da, na))

ne_ge_st = OnePiece.io.table.read(joinpath(ou, na))

fe_nu = Dict(fe => sc for (fe, sc) in eachrow(ne_ge_st))

println("Gene", "\t", "Old", "\t", "New")

di = 4

n_mi = 0

for (fe, ol) in eachrow(ol_ge_st)

    ol = round(ol; digits = di)

    nu = round(fe_nu[fe]; digits = di)

    if ol != nu

        println(fe, "\t", ol, "\t", nu)

        n_mi += 1

    end

end

n_fe = size(ol_ge_st, 1)

mi = n_mi / n_fe

println("Missed ", n_mi, "/", n_fe, " scores (", round(mi * 100; digits = di), "%).")

@assert mi < 0.001

---

In [ ]:
if isdir(TE)

    rm(TE, recursive = true)

    println("Removed ", TE, ".")

end